In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import requests

from shapely.geometry import Point, MultiPoint
import hashlib

import geopandas as gpd
from shapely.geometry import Point, LineString

In [1]:
ruta_est_coord = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\6. MIBICI\estaciones_coordenadas_240718.xlsx"
ruta_viajes = r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\Red_MiBici\Viajes\2403.csv"

ruta_viajes_coord = r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\Red_MiBici\Viajes\2403_coord.xlsx"

output_gpkg_file = r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\Red_MiBici\OD\OD_MiBici_24.gpkg"
capa = 'OD_2403'
capa_simple = 'OD_2403_Simple'

# TRATAR ESTACIONES

In [45]:
shape_file = r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\Red_MiBici\Estaciones 2407\360 estaciones MiBici.shp"
csv = r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\Red_MiBici\Estaciones 2407\nomenclatura_2024_06.csv"

ruta_est_coord = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\6. MIBICI\estaciones_coordenadas_240718.xlsx"
epsg_code = 6368

In [46]:
# Leer la capa shape
gdf = gpd.read_file(shape_file)

# Proyectar a EPSG:6368
gdf = gdf.to_crs(epsg=epsg_code)

#Leer csv
df = pd.read_csv(csv, encoding='latin-1')

# Merge con el DataFrame de centroides para añadir coordenadas de origen y destino
gdf = gdf.merge(df[['obcn','id']], how='left', left_on='CLAVE', right_on='obcn').rename(columns={'X': 'X_origen', 'Y': 'Y_origen'}).drop(columns=['obcn'])

#Filtrar columnas necesarias
gdf= gdf[['id','CLAVE','Cruce_C','ETAPA','Exten_m','geometry']]

def multipoint_to_point(multipoint):
    if isinstance(multipoint, MultiPoint):
        points = list(multipoint.geoms)
        if len(points) == 1:
            return points[0]
    return multipoint

gdf['geometry'] = gdf['geometry'].apply(multipoint_to_point)

# Extract x and y coordinates
gdf['X'] = gdf.geometry.apply(lambda geom: geom.x)
gdf['Y'] = gdf.geometry.apply(lambda geom: geom.y)

# Exportar a Excel
gdf.to_excel(ruta_est_coord, index=False)

In [ ]:
#estaciones ara

In [13]:
shape_file = r"C:\Users\UTIC\Downloads\Viajes Marzo\Viajes Marzo\Mi_BICI24.shp"
csv = r"C:\Users\UTIC\Documents\Alejandro\Docs\DATA\Red_MiBici\Estaciones 2407\nomenclatura_2024_06.csv"

ruta_est_coord = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\6. MIBICI\estaciones_coordenadas_240807.xlsx"
epsg_code = 6368

In [14]:
# Leer la capa shape
gdf = gpd.read_file(shape_file)

# Proyectar a EPSG:6368
gdf = gdf.to_crs(epsg=epsg_code)

#Leer csv
df = pd.read_csv(csv, encoding='latin-1')

In [15]:
# Limpiar los espacios en blanco al inicio y al final de los valores en la columna 'Punto'
df['obcn'] = df['obcn'].str.strip()
gdf['CLAVE'] = gdf['CLAVE'].str.strip()

# Merge con el DataFrame de centroides para añadir coordenadas de origen y destino
gdf = gdf.merge(df[['obcn','id']], how='left', left_on='CLAVE', right_on='obcn').rename(columns={'X': 'X_origen', 'Y': 'Y_origen'}).drop(columns=['obcn'])

#Filtrar columnas necesarias
gdf= gdf[['id','CLAVE','Cruce_C','ETAPA','Exten_m','geometry']]

def multipoint_to_point(multipoint):
    if isinstance(multipoint, MultiPoint):
        points = list(multipoint.geoms)
        if len(points) == 1:
            return points[0]
    return multipoint

gdf['geometry'] = gdf['geometry'].apply(multipoint_to_point)

# Extract x and y coordinates
gdf['X'] = gdf.geometry.apply(lambda geom: geom.x)
gdf['Y'] = gdf.geometry.apply(lambda geom: geom.y)

# Exportar a Excel
gdf.to_excel(ruta_est_coord, index=False)

# Unir coordenadas desde estaciones a viajes

In [77]:
#excel con viajes mibici
df=pd.read_csv(ruta_viajes, encoding='latin-1') 

#excel con cp y coordenadas
df_est_coord =pd.read_excel(ruta_est_coord)

# Merge con el DataFrame de centroides para añadir coordenadas de origen y destino
df = df.merge(df_est_coord[['id','X','Y']], how='left', left_on='Id de estación de inicio', right_on='id').rename(columns={'X': 'X_origen', 'Y': 'Y_origen'}).drop(columns=['id'])
df = df.merge(df_est_coord[['id','X','Y']], how='left', left_on='Id de estación de fin de viaje', right_on='id').rename(columns={'X': 'X_destino', 'Y': 'Y_destino'}).drop(columns=['id'])

# Exportar a Excel
df.to_excel(ruta_viajes_coord, index=False)

# ORIGEN Y DESTINO

In [78]:
#excel con OD emca con coordenadas
df=pd.read_excel(ruta_viajes_coord) 

# Crear geometrías de líneas
df['geometry'] = df.apply(lambda row: LineString([(row['X_origen'], row['Y_origen']), (row['X_destino'], row['Y_destino'])]), axis=1)

# Crear GeoDataFrame de líneas
gdf_lines = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:6368')

# # Mantener solo las columnas necesarias
# gdf_lines = gdf_lines[['geometry'] + [col for col in df.columns if not col.startswith('geometry_')]]

# Exportar a geopackage
gdf_lines.to_file(output_gpkg_file, layer=capa, driver='GPKG')

# Agrupar líneas iguales y contar cantidad de viajes
gdf_lines['count'] = 1

# Crear una función para generar un hash basado en la geometría de la línea
def geometry_hash(geometry):
    return hashlib.sha256(geometry.wkb).hexdigest()

# Crear una nueva columna con el hash de la geometría
gdf_lines['geometry_hash'] = gdf_lines['geometry'].apply(geometry_hash)

# Agrupar por el hash de la geometría y contar el número de ocurrencias
gdf_grouped = gdf_lines.groupby('geometry_hash').agg({
    'geometry': 'first',
    'count': 'size'
}).reset_index()

# Renombrar la columna 'count' a 'NUMERO'
gdf_grouped = gdf_grouped.rename(columns={'count': 'NUMERO'})

# Convertir a GeoDataFrame
gdf_grouped = gpd.GeoDataFrame(gdf_grouped, geometry='geometry', crs='EPSG:6368')

# Exportar a geopackage
gdf_grouped.to_file(output_gpkg_file, layer=capa_simple, driver='GPKG')